In [1]:
import os,sys
sys.path.append("%s/analysis-CommonTools/"%os.getenv('HOME'))
from common_imports import *
from matplotlib_tools import *

In [2]:
import ROOT

Welcome to JupyROOT 6.26/00


In [3]:
ROOT.gStyle.SetOptStat(0)

In [4]:
%jsroot on

In [5]:
do_iev_check = False

lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)

hadron_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV

In [6]:
c = ROOT.TCanvas()

In [7]:
gst_files_dir = "/Users/wketchum/Data/LDMX/eN_Ti_Base"
gst_files = ROOT.TFileCollection()
for run in range(11):
    gst_files.Add(f"{gst_files_dir}/run_{run}_gntp.0.gst.root")    

In [8]:
gst_chain = ROOT.TChain("gst");
gst_chain.AddFileInfoList(gst_files.GetList())

1

In [9]:
knob_names = ["FrCEx_N","FrAbs_N","FrInel_N","FrPiProd_N",#"MFP_N",
              "FrCEx_pi","FrAbs_pi","FrInel_pi","FrPiProd_pi"]

In [10]:
for kn in knob_names:
    wfiles = ROOT.TFileCollection()
    for run in range(11):
        rstring = "run_%d_gntp"%run
        wfiles.Add(f"/Users/wketchum/Data/LDMX/eN_Ti_Rw_FSI/{rstring}/run_{run}_gntp_weights_{kn}.root")
    for twk in [0,1,3,4]:
        wtree = ROOT.TChain(f"{kn}_twk{twk}")
        wtree.AddFileInfoList(wfiles.GetList())
        if(gst_chain.GetEntries()!=wtree.GetEntries()):
            print(f"Error! Number of entries not equal: {gst_chain.GetEntries()}!={wtree.GetEntries()}")
            break
        sigmaval=twk-2
        sigmasign="p" if sigmaval>0 else "n"
        gst_chain.AddFriend(wtree,f"wght_{kn}_{sigmasign}{abs(sigmaval)}")

In [11]:
df_gst_all = ROOT.RDataFrame(gst_chain)

In [12]:
for kn in knob_names:
    for sigmastr in ["n2","n1","p1","p2"]:
        for var in ["iev","wght"]:
            df_gst_all = df_gst_all.Alias(f"wght_{kn}_{sigmastr}_{var}",f"wght_{kn}_{sigmastr}.{var}")

In [13]:
#if we want to make sure that everything was in order...

if do_iev_check:
    for kn in knob_names:
        for sigmastr in ["n2"]:# can afford to just do this once... ,"n1","p1","p2"]:
            print(f"Checking knob {kn}_{sigmastr} for mismatches")
            n_mismatch = df_gst_all.Filter(f"iev!=wght_{kn}_{sigmastr}_iev").Count().GetValue()
            if(n_mismatch!=0):
                print(f"Error! Mismatch in iev for {kn}_{sigmastr}")

In [14]:
df_gst_all = df_gst_all.Define("ptl","sqrt(pxl*pxl+pyl*pyl)")
df_gst_all = df_gst_all.Define("thetazl","atan2(ptl,pzl)")

In [15]:
df_gst_all = df_gst_all.Define("thetaxzf","atan2(pxf,pzf)")
df_gst_all = df_gst_all.Define("thetayzf","atan2(pyf,pzf)")
df_gst_all = df_gst_all.Define("ptf","sqrt(pxf*pxf+pyf*pyf)")
df_gst_all = df_gst_all.Define("thetazf","atan2(ptf,pzf)")

In [16]:
df_gst_all = df_gst_all.Define("massf","sqrt(Ef*Ef-pf*pf)")
df_gst_all = df_gst_all.Define("kef","Ef-massf")

In [17]:
df_gst_all = df_gst_all.Define("energy_transfer","Ev-El")

In [18]:
hadronf_vars = ["pdgf","Ef","pxf","pyf","pzf","ptf","pf","massf","kef","thetaxzf","thetayzf","thetazf"]

In [19]:
for var in hadronf_vars:
    df_gst_all = df_gst_all.Define(f"{var}_p",f"{var}[pdgf==2212]")
    df_gst_all = df_gst_all.Define(f"{var}_n",f"{var}[pdgf==2112]")
    df_gst_all = df_gst_all.Define(f"{var}_pi0",f"{var}[pdgf==111]")
    df_gst_all = df_gst_all.Define(f"{var}_piplus",f"{var}[pdgf==211]")
    df_gst_all = df_gst_all.Define(f"{var}_piminus",f"{var}[pdgf==-211]")
    
    df_gst_all = df_gst_all.Define(f"{var}_p_accept",f"{var}[pdgf==2212 && thetazf<{hadron_accept_angle} && kef>{hadron_accept_ke}]")
    df_gst_all = df_gst_all.Define(f"{var}_n_accept",f"{var}[pdgf==2112 && thetazf<{hadron_accept_angle} && kef>{hadron_accept_ke}]")
#    df_gst_all = df_gst_all.Define(f"{var}_pi0_accept",f"{var}_pi0[pdgf==111 && thetazf<{hadron_accept_angle} && kef>{hadron_accept_ke}]")
    df_gst_all = df_gst_all.Define(f"{var}_piplus_accept",f"{var}[pdgf==211 && thetazf<{hadron_accept_angle} && kef>{hadron_accept_ke}]")
    df_gst_all = df_gst_all.Define(f"{var}_piminus_accept",f"{var}[pdgf==-211 && thetazf<{hadron_accept_angle} && kef>{hadron_accept_ke}]")

In [ ]:
for n in df_gst_all.GetColumnNames():
    print(n)

In [ ]:
etransfer_bins = array.array('d',np.logspace(np.log10(0.05),np.log10(4),100).tolist())
q2transfer_bins = array.array('d',np.logspace(np.log10(0.02),np.log10(10),100).tolist())

In [ ]:
h_lep_q2vw = df_gst_all.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,len(q2transfer_bins)-1,q2transfer_bins),
                       "energy_transfer","Q2")

In [ ]:
h_lep_q2vw.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [21]:
df_gst_lep_accept = df_gst_all.Filter(f"ptl>0.4 && abs(thetazl)<{radians(40)}")

In [ ]:
h_lep_q2vw_accept = df_gst_lep_accept.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,
                                            len(q2transfer_bins)-1,q2transfer_bins),
                                      "energy_transfer","Q2")

In [ ]:
h_lep_q2vw_accept.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [ ]:
h = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"kef_p_accept")
h2 = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"kef_p")

In [ ]:
h.Draw()
h2.SetLineColor(ROOT.kRed)
h2.Draw("same")
c.Draw()

In [ ]:
h3 = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"ptf_piplus")

In [ ]:
h3.Draw()
c.Draw()

In [22]:
h4 = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"pf_piplus")
h4.Draw()
c.Draw()

In [23]:
h4.SetTitle("#pi^{+};Momentum (GeV/c);")
h4.Draw()
c.Draw()